In [47]:
import uuid
import random
import pygad
import numpy as np
random.seed(42)

In [48]:
class Room:
    def __init__(self, name, width, height, pixels):
        self.name = name
        self.width = width
        self.height = height
        self.pixels = pixels

    def __str__(self):
        room_str = ""
        for row in self.pixels:
            room_str += "".join(row) + "\n"
        return room_str

# Example usage
a = Room("A", 6, 6, [
    [' #', ' #', ' #', ' #', ' #', ' #'],
    [' #', ' +', ' +', ' +', ' +', ' #'],
    [' #', ' +', ' A', ' A', ' +', ' #'],
    [' #', ' +', ' A', ' A', ' +', ' #'],
    [' #', ' +', ' +', ' +', ' +', ' #'],
    [' #', ' #', ' #', ' #', ' #', ' #']
])

b = Room("B", 4, 6, [
    [' #', ' #', ' #', ' #'],
    [' #', ' +', ' +', ' #'],
    [' #', ' B', ' B', ' #'],
    [' #', ' B', ' B', ' #'],
    [' #', ' +', ' +', ' #'],
    [' #', ' #', ' #', ' #']
])

c = Room("C", 6, 4, [
    [' #', ' #', ' #', ' #', ' #', ' #'],
    [' #', ' +', ' C', ' C', ' +', ' #'],
    [' #', ' +', ' C', ' C', ' +', ' #'],
    [' #', ' #', ' #', ' #', ' #', ' #']
])

d = Room("D", 4, 4, [
    [' #', ' #', ' #', ' #'],
    [' #', ' D', ' D', ' #'],
    [' #', ' D', ' D', ' #'],
    [' #', ' #', ' #', ' #']
])

room_map = {
    0: a,
    1: b,
    2: c,
    3: d
}


In [49]:
class Layout:
    def __init__(self, layout_id, layout, side=16):
        self.id = layout_id
        self.layout = layout
        self.height = len(layout)
        self.width = max(len(row) for row in layout)
        self.side = side
        w = self.width if self.width <= self.side else self.side
        h = self.height if self.height <= self.side else self.side
        self.area = (w * h) -  sum(row.count('  ') for row in layout)

    def calculate_score(self, side=16):
        self.max_score = side**2
        penalty = 0  
        
        # Subtract any filled space beyond side width from max_score
        if (self.width > side):
            # calculate the area of the layout beyond column 16
            w_area = (self.width - side) * self.height
            # count all the empty spaces from column 16 onwards
            w_empty = 0
            for row in self.layout:
                w_empty += row[side:].count('  ')
            w_area -= w_empty
            print(f'Width penalty: {w_area}')
            penalty += w_area

        # Subtract any filled space beyond side height from max_score
        if (self.height > side):
            # calculate the area of the layout beyond row 16
            h_area = (self.height - side) * self.width
            # count all the empty spaces from row 16 onwards
            h_empty = 0
            for row in self.layout[side:]:
                h_empty += row.count('  ')
            h_area -= h_empty
            print(f'Height penalty: {h_area}')
            penalty += h_area


        self.score = self.area - penalty
        return self.score

    def draw_layout(self):
        print(f"\nLayout: {self.id}\n")
        for row in self.layout:
            print("".join(row))
        print(f"\nLayout area: {self.area}, Maximum area: {self.side**2}, Width: {self.width}, Height: {self.height}\n")
        if self.score is not None:
            print(f"Score: {self.score}\n")
        print("-" * 50)


SyntaxError: f-string: invalid syntax (1060744163.py, line 48)

In [ ]:
class RoomGenerator:
    def __init__(self, room_map):
        self.room_map = room_map
        self.layout = []

    def generate_layout(self, layout_id, genotype):
        if len(genotype) % 2 != 0:
            raise ValueError("Genotype must be an even number")
        self.id = layout_id
        self.layout = []  # Reset layout for each new phenotype
        x, y = 0, 0  # Starting position
        prev_room = Room("Empty", 0, 0, [['  ']])

        for i in range(0, len(genotype), 2):
            room_type = genotype[i]
            direction = genotype[i + 1]
            if room_type not in self.room_map:
                raise ValueError(f"Room type {room_type} not defined. Select room types 0-3")
            if direction not in range(4):
                raise ValueError(f"Direction {direction} not defined. Select directions 0-3")            
            room = self.room_map[room_type]

            if i == 0:
                # Add the first room to the layout
                self.layout.append((x, y, room))
                prev_room = room
                continue
            # Adjust position based on direction
            if direction == 0: # Right
                x += prev_room.width - 1
            elif direction == 1:  # Down
                y += prev_room.height - 1
            elif direction == 2:  # Left
                x -= room.width - 1
            elif direction == 3:  # Up
                y -= room.height - 1

            # Add the room to the layout list
            self.layout.append((x, y, room))
            prev_room = room
        return self.layout

    def map_layout_to_grid(self):
            # Set all the layout to positive coordinates
            min_x = min([x for x, _, _ in self.layout])
            min_y = min([y for _, y, _ in self.layout])
            for i in range(len(self.layout)):
                x, y, room = self.layout[i]
                self.layout[i] = (x - min_x, y - min_y, room)

            # Find the dimensions of the layout
            max_x = max([x + room.width for x, _, room in self.layout])
            max_y = max([y + room.height for _, y, room in self.layout])

            # Initialize the grid
            layout_grid = [['  ' for _ in range(max_x)] for _ in range(max_y)]

            # Map the rooms to the grid
            for x, y, room in self.layout:
                # Check if the indices are within bounds
                if 0 <= y < max_y and 0 <= x < max_x:
                    for row in range(room.height):
                        for col in range(room.width):
                            # Dynamically adjust grid size if necessary
                            if y + row >= len(layout_grid):
                                layout_grid.append(['  ' for _ in range(max_x)])

                            if x + col >= len(layout_grid[y + row]):
                                layout_grid[y + row].append('  ')

                            # Check if the current pixel is a wall in both the room and the existing layout
                            if room.pixels[row][col] == '#' and layout_grid[y + row][x + col] == '#':
                                # Combine walls into a single pixel width
                                layout_grid[y + row][x + col] = ' #'
                            else:
                                # Otherwise, place the room pixel onto the grid
                                layout_grid[y + row][x + col] = room.pixels[row][col]

            # Create a Grid instance
            layout_instance = Layout(self.id, layout_grid)

            return layout_instance


In [ ]:
room_generator = RoomGenerator(room_map)

# wide test
#test = [2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0]
#test = [1, 3, 1, 1, 1, 3, 1, 1, 1, 3, 1, 1, 1, 3, 1, 1, 1, 3, 1, 1]
#test = [1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3]
test = [0, 0, 1, 1, 2, 2, 3, 3]
test_layout = room_generator.generate_layout("Demo", test)
test_grid = room_generator.map_layout_to_grid()
test_grid.calculate_score()
test_grid.draw_layout()

''' 
pheno_1 = [0, 3, 3, 2, 2, 1, 1, 0, 3, 1, 3, 2, 0, 3, 2, 0, 1, 1]
result_layout_1 = room_generator.generate_layout(uuid.uuid1(), pheno_1)
grid_1 = room_generator.map_layout_to_grid()
grid_1.draw_layout()

pheno_2 = [1, 0, 3, 1, 3, 2, 0, 3, 1, 0, 2, 1, 3, 1, 0, 0, 2, 3]
result_layout_2 = room_generator.generate_layout(uuid.uuid1(), pheno_2)
grid_2 = room_generator.map_layout_to_grid()
grid_2.draw_layout()

pheno_3 = [2, 3, 1, 2, 0, 1, 3, 0, 2, 0, 1, 2, 3, 1, 2, 0, 3, 3]
result_layout_3 = room_generator.generate_layout(uuid.uuid1(), pheno_3)
grid_3 = room_generator.map_layout_to_grid()
grid_3.draw_layout()

pheno_4 = [3, 2, 0, 3, 1, 0, 2, 1, 2, 2, 1, 1, 0, 3, 2, 1, 1, 2]
result_layout_4 = room_generator.generate_layout(uuid.uuid1(), pheno_4)
grid_4 = room_generator.map_layout_to_grid()
grid_4.draw_layout()
'''



Layout: Demo

           # # # # # #
           # + + + + #
 # # # #   # + A A + #
 # D D #   # + A A + #
 # D D #   # + + + + #
 # # # # # # # # # # #
 # + C C + # + + #    
 # + C C + # B B #    
 # # # # # # B B #    
           # + + #    
           # # # #    

Area: 88, Width: 11, Height: 11

Score: 88

--------------------------------------------------


' \npheno_1 = [0, 3, 3, 2, 2, 1, 1, 0, 3, 1, 3, 2, 0, 3, 2, 0, 1, 1]\nresult_layout_1 = room_generator.generate_layout(uuid.uuid1(), pheno_1)\ngrid_1 = room_generator.map_layout_to_grid()\ngrid_1.draw_layout()\n\npheno_2 = [1, 0, 3, 1, 3, 2, 0, 3, 1, 0, 2, 1, 3, 1, 0, 0, 2, 3]\nresult_layout_2 = room_generator.generate_layout(uuid.uuid1(), pheno_2)\ngrid_2 = room_generator.map_layout_to_grid()\ngrid_2.draw_layout()\n\npheno_3 = [2, 3, 1, 2, 0, 1, 3, 0, 2, 0, 1, 2, 3, 1, 2, 0, 3, 3]\nresult_layout_3 = room_generator.generate_layout(uuid.uuid1(), pheno_3)\ngrid_3 = room_generator.map_layout_to_grid()\ngrid_3.draw_layout()\n\npheno_4 = [3, 2, 0, 3, 1, 0, 2, 1, 2, 2, 1, 1, 0, 3, 2, 1, 1, 2]\nresult_layout_4 = room_generator.generate_layout(uuid.uuid1(), pheno_4)\ngrid_4 = room_generator.map_layout_to_grid()\ngrid_4.draw_layout()\n'

In [ ]:
def generate_random_genotypes(individuals, sequence_length):
    population = []
    for _ in range(individuals):
        population.append([random.randint(0, 3) for _ in range(sequence_length)])
    return population

In [ ]:
pop = generate_random_genotypes(20, 20)
for individual in pop:
    print(individual)

[0, 0, 2, 1, 1, 1, 0, 0, 3, 0, 0, 0, 1, 1, 0, 1, 3, 1, 3, 2]
[0, 1, 3, 2, 2, 1, 1, 2, 0, 0, 3, 0, 2, 2, 2, 0, 3, 0, 3, 0]
[2, 2, 1, 0, 0, 1, 2, 0, 1, 0, 3, 2, 3, 2, 1, 2, 2, 1, 2, 0]
[1, 1, 1, 3, 3, 2, 1, 2, 0, 1, 0, 2, 3, 2, 0, 1, 2, 1, 3, 3]
[3, 1, 2, 1, 1, 2, 3, 3, 2, 1, 1, 3, 0, 0, 0, 1, 1, 3, 0, 3]
[3, 3, 2, 0, 0, 2, 2, 0, 2, 3, 1, 3, 0, 2, 1, 0, 2, 1, 1, 2]
[1, 0, 2, 3, 0, 0, 2, 2, 1, 0, 1, 0, 0, 3, 0, 1, 1, 3, 1, 2]
[3, 1, 1, 2, 3, 2, 3, 3, 0, 1, 1, 0, 2, 0, 1, 1, 0, 0, 0, 1]
[0, 0, 2, 0, 1, 2, 3, 1, 1, 3, 1, 3, 3, 1, 0, 0, 3, 2, 3, 3]
[3, 0, 0, 0, 3, 2, 0, 1, 1, 1, 3, 1, 3, 1, 2, 3, 1, 0, 3, 0]
[0, 0, 0, 1, 1, 3, 3, 3, 1, 3, 0, 1, 3, 0, 3, 2, 3, 2, 3, 3]
[1, 1, 2, 1, 0, 0, 2, 0, 0, 3, 1, 0, 0, 1, 0, 0, 1, 3, 0, 1]
[0, 0, 3, 2, 2, 1, 2, 1, 2, 3, 1, 2, 3, 2, 0, 0, 3, 0, 0, 1]
[2, 1, 2, 0, 1, 2, 2, 1, 3, 2, 0, 2, 0, 1, 2, 0, 0, 1, 2, 2]
[1, 2, 1, 2, 3, 2, 0, 0, 3, 2, 0, 0, 2, 1, 2, 1, 3, 3, 0, 0]
[0, 1, 0, 2, 1, 3, 1, 0, 2, 2, 0, 2, 1, 1, 0, 2, 3, 1, 1, 1]
[1, 3, 0, 1, 2, 3, 1, 2,